![rmotr](https://user-images.githubusercontent.com/7065401/52071918-bda15380-2562-11e9-828c-7f95297e4a82.png)
<hr style="margin-bottom: 20px;">
<img src="https://user-images.githubusercontent.com/7065401/75165824-badf4680-5701-11ea-9c5b-5475b0a33abf.png"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Lecture de données externes & Visualisation

[Source](https://blockchain.info/charts/market-price)

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Pratique !

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Pandas peut facilement lire des données stockées dans différents formats de fichiers comme CSV, JSON, XML ou même Excel. L'analyse nécessite toujours de spécifier la structure correcte, l'encodage et d'autres détails. La méthode `read_csv` lit les fichiers CSV et accepte de nombreux paramètres.

In [63]:
# pd.read_csv?

In [64]:
df = pd.read_csv("btc-market-price.csv")

In [138]:
df.head()

Le fichier CSV que nous lisons n'a que deux colonnes : `timestamp` et `price`. Il n'a pas d'en-tête, il contient des espaces et a des valeurs séparées par des virgules. pandas a automatiquement attribué la première ligne de données comme en-têtes, ce qui est incorrect. Nous pouvons remplacer ce comportement avec le paramètre `header` :

In [139]:
df = pd.read_csv("btc-market-price.csv", header=None)

In [140]:
df.head()

Nous pouvons ensuite définir explicitement les noms de chaque colonne en définissant l'attribut `df.columns` :

In [68]:
df.columns = ['Timestamp', 'Price']

In [141]:
df.shape

In [142]:
df.head()

In [143]:
df.tail(3)

In [144]:
df.dtypes

Le type de la colonne `Price` a été correctement interprété comme `float`, mais le `Timestamp` a été interprété comme une chaîne régulière (`object` en notation pandas) :

Nous pouvons effectuer une opération vectorisée pour analyser toutes les valeurs Timestamp en tant qu'objets `Datetime` :

In [145]:
pd.to_datetime(df['Timestamp']).head()

In [146]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [147]:
df.head()

In [148]:
df.dtypes

Le timestamp ressemble beaucoup à l'index de ce `DataFrame` : `date > prix`. Nous pouvons changer l'ID auto-incrémental généré par pandas et utiliser la colonne `Timestamp DS` comme Index :

In [77]:
df.set_index('Timestamp', inplace=True)

In [149]:
df.head()

In [150]:
df.loc['2017-09-29']

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Tout assembler
Et maintenant, nous sommes enfin arrivés à la version finale et souhaitée du `DataFrame` analysé à partir de notre fichier CSV. Les étapes étaient :

In [151]:
df = pd.read_csv("btc-market-price.csv", header=None)
df.columns = ['Timestamp', 'Price']
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)

In [152]:
df.head()

**Il devrait y avoir une meilleure façon**. Et il y en a une. Et il y en a généralement une, explicitement avec toutes ces tâches répétitives avec pandas.

La fonction `read_csv` est extrêmement puissante et vous pouvez spécifier beaucoup plus de paramètres au moment de l'importation. Nous pouvons obtenir les mêmes résultats avec une seule ligne en faisant :

In [82]:
df = pd.read_csv(
    "btc-market-price.csv",
    header=None,
    names=['Timestamp', 'Price'],
    index_col=0,
    parse_dates=True
)

In [153]:
df.head()

In [84]:
df.loc['2017-09-29']

Price    4193.574667
Name: 2017-09-29 00:00:00, dtype: float64

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Bases de la visualisation

`pandas` s'intègre avec Matplotlib et créer un graphique est aussi simple que :

In [154]:
df.plot()

En coulisses, il utilise l'interface de `matplotlib.pyplot`. Nous pouvons créer un graphique similaire avec la fonction `plt.plot()` :

In [155]:
plt.plot(df.index, df['Price'])

`plt.plot()` accepte de nombreux paramètres, mais les deux premiers sont les plus importants : les valeurs pour les axes `X` et `Y`. Un autre exemple :

In [156]:
x = np.arange(-10, 11)

In [158]:
plt.plot(x, x ** 2)

Nous utilisons l'API globale de `matplotlib`, qui est horrible mais c'est la plus populaire. Nous apprendrons plus tard comment utiliser l'API _orientée objet_ qui rendra notre travail beaucoup plus facile.

In [96]:
plt.plot(x, x ** 2)
plt.plot(x, -1 * (x ** 2))

Chaque fonction `plt` modifie l'état global. Si vous voulez définir les paramètres de votre graphique, vous pouvez utiliser la fonction `plt.figure`. D'autres comme `plt.title` continuent de modifier le graphique global :

In [159]:
plt.figure(figsize=(12, 6))
plt.plot(x, x ** 2)
plt.plot(x, -1 * (x ** 2))
plt.show()
plt.title('My Nice Plot')

Certains des arguments dans `plt.figure` et `plt.plot` sont disponibles dans l'interface `plot` de pandas :

In [160]:
df.plot(figsize=(16, 9), title='Bitcoin Price 2017-2018')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Une analyse plus difficile

Pour démontrer la visualisation de deux colonnes ensemble, nous allons essayer d'ajouter les prix d'Ether à notre DataFrame `df`. Les données de prix ETH se trouvent dans le fichier `data/eth-price.csv`. Le problème est qu'il semble que ce fichier CSV ait été créé par quelqu'un qui détestait vraiment les programmeurs. Jetez-y un œil et voyez à quel point il est moche. Nous utiliserons quand même `pandas` pour l'analyser.

In [113]:
eth = pd.read_csv(
    "eth-price.csv",
    index_col=0,
    parse_dates=True
)


In [161]:
eth.head()

Comme vous pouvez le voir, il a une colonne `Value` (qui représente le prix), une colonne `Date(UTC)` qui a une chaîne représentant les dates et aussi une date `UnixTimeStamp` représentant la date et l'heure au format timestamp unix. L'en-tête est lu automatiquement, essayons d'analyser les dates avec le lecteur CSV :

In [162]:
print(eth.dtypes)

Il semble que l'attribut `parse_dates` n'ait pas fonctionné. Nous devrons ajouter un peu plus de personnalisation. Divisons ce problème et concentrons-nous d'abord sur le problème de "l'analyse des dates". L'option la plus simple serait d'utiliser la colonne `UnixTimeStamp`. Le module `pandas` a une fonction `to_datetime` qui convertit automatiquement les timestamps Unix en objets Datetime :

In [163]:
pd.to_datetime(eth['UnixTimeStamp']).head()

Le problème est la précision des timestamps unix. Pour faire correspondre les deux colonnes, nous devrons utiliser le même index et, notre `df` contenant les prix Bitcoin, est "par jour" :

In [164]:
df.head()

Nous pourrions soit supprimer la précision de `UnixTimeStamp`, soit essayer d'analyser `Date(UTC)`. Faisons l'analyse de chaîne de `Date(UTC)` pour le plaisir :

In [165]:
pd.to_datetime(eth['Date(UTC)']).head()

Cela semble bien fonctionner ! Pourquoi ne parse-t-il pas alors la colonne `Date(UTC)` ? Simple, le paramètre `parse_dates=True` demandera à pandas d'analyser l'index du `DataFrame`. Si vous voulez analyser une autre colonne, vous devez explicitement passer la position ou le nom de la colonne :

In [166]:
pd.read_csv('eth-price.csv', parse_dates=[0]).head()

Tout assembler à nouveau :

In [167]:
eth = pd.read_csv("eth-price.csv", parse_dates=[0], index_col=0)
print(eth.info())

eth.head()

Nous pouvons maintenant combiner les deux `DataFrame` en un seul. Les deux ont le même index, donc aligner les deux prix sera facile. Créons d'abord un `DataFrame` vide avec l'index des prix Bitcoin :

In [125]:
prices = pd.DataFrame(index=df.index)

In [168]:
prices.head()

Et nous pouvons maintenant simplement définir des colonnes à partir des autres `DataFrame` :

In [127]:
prices['Bitcoin'] = df['Price']

In [128]:
prices['Ether'] = eth['Value']

In [169]:
prices.head()

Nous pouvons maintenant essayer de tracer les deux valeurs :

In [172]:
prices.plot(figsize=(12, 6))

il semble qu'il y ait un petit écart entre décembre 2017 et janvier 2018. Zoomons là :

In [174]:
prices.loc['2017-12-01':'2018-01-01'].plot(figsize=(12, 6), label = "Dec to Jan")

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)